In [ ]:
#importar as bibliotecas
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
nltk.download('stopwords')
stoplist = set(stopwords.words("english"))
print(stopwords.words('english'))

In [ ]:
#carregar dataset
#os dados foram carregados do kaggle para treinar nossa ia.
df = pd.read_csv('train.csv',encoding='utf8')

In [ ]:
df.isnull().sum()
#retirar nulos
df.dropna(inplace=True)
df.isnull().sum()

In [ ]:
#juntar os campos
df['all'] = df['title'] +'  '+ df['author'] +'  '+ df['text']  
pd.options.display.max_colwidth = 150
print(df['all'])

In [ ]:
#criar função de stemmer
#ele tenta trazer a palavra até a palavra rais, ou seja, se a palavra for atuando ele tenta trazer ela para atuar.
porter_stemmer = PorterStemmer()
 
def stemming_tokenizer(df):

    words= re.sub(r"[^A-Za-z]", " ", df).lower().split()
    words= [porter_stemmer.stem(word) for word in words if not words in stopwords.words('english')]
    words= ' '.join(words)
    return words 

In [ ]:

df['features'] = df['all'].apply(stemming_tokenizer)
print(df['features'])

In [ ]:
#separa em features e label
#label representa a veracidade da informação sendo 1 para fake e 0 pra real
x = df['features'].values
y = df['label'].values

x.size

In [ ]:
# convert de texto para numeros
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
z = vectorizer.transform(x)


In [ ]:
z_train, z_test, y_train, y_test = train_test_split(z, y, test_size=0.2,stratify=y, random_state=42) 

In [ ]:
print(len(y_test))

In [ ]:
#treinando o modelo

lr = LogisticRegression()
lr.fit(z_test,y_test)

In [ ]:
#previsão 

pred = lr.predict(z_test)

acc = accuracy_score(y_test,pred)
f'Acurácia:{acc * 100:.2f}'


In [ ]:
# Pré-processamento da notícia
noticia = "Rapper T.I.: Trump a ’Poster Child For White Supremacy’  Jerome Hudson  Rapper T. I. unloaded on black celebrities who met with Donald Trump afte"
noticia_preprocessada = stemming_tokenizer(noticia)

# Convertendo a notícia em uma representação numérica
noticia_numerica = vectorizer.transform([noticia_preprocessada])

#Fazendo a previsão
probabilidade_falsa = lr.predict_proba(noticia_numerica)[:, 1]  # Obtém a probabilidade da classe falsa

chance_falsa = probabilidade_falsa[0] * 100  # Multiplica por 100 para obter a porcentagem
chance_verdadeira = 100 - chance_falsa  # Calcula a porcentagem de chances de ser verdadeira

if chance_falsa > 50:
    print("A notícia é falsa.")
else:
    print("A notícia é verdadeira.")

